<h4> Import Dependencies </h4>

In [229]:
import os
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
from pprint import pprint
import sqlite3
import csv

# API keys
from config import api_key

<h2>Financial Data, API Exctraction</h2>

<h4>Make API Calls and Populate a DataFrame</h4>

In [230]:
#Starting Year
year = 2000

In [231]:
#API url path
url = f"https://api.open.fec.gov/v1/elections/?api_key={api_key}&sort=-total_receipts&sort_hide_null=false&office=president&election_full=true&sort_null_only=false&cycle={year}&sort_nulls_last=false&page=1"

# read data from the API 
response = requests.get(url).json()
# pprint(response)

# convert JSON to dataframe
candidate_finance = pd.json_normalize(response, "results")
# print(candidate_finance.shape)

#Append following year data to existing DataFrame
for yr in range(year + 4, 2017, 4):
    url = f"https://api.open.fec.gov/v1/elections/?api_key={api_key}&sort=-total_receipts&sort_hide_null=false&office=president&election_full=true&sort_null_only=false&cycle={yr}&sort_nulls_last=false&page=1"
    response = requests.get(url).json()
   
    sub_df = pd.json_normalize(response, "results")
    candidate_finance.head(2)
#     print(f"{yr} {type(sub_df)}  ================================")
    candidate_finance = candidate_finance.append(sub_df, ignore_index=True)
    
# Format dataframe
candidate_finance = candidate_finance[['candidate_election_year', 'candidate_id', 'candidate_name', 'party_full', 'total_receipts', 'total_disbursements', 'cash_on_hand_end_period']]
candidate_finance.style.format({"total_receipts": '${0:,.2f}',
                                "total_disbursements": '${0:,.2f}',
                                "cash_on_hand_end_period": '${0:,.2f}',})

candidate_finance["candidate_name"]
pprint(response)

{'api_version': '1.0',
 'pagination': {'count': 669, 'page': 1, 'pages': 34, 'per_page': 20},
 'results': [{'candidate_election_year': 2016,
              'candidate_id': 'P00003392',
              'candidate_name': 'CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL '
                                'KAINE',
              'candidate_pcc_id': 'C00575795',
              'candidate_pcc_name': 'HILLARY FOR AMERICA',
              'cash_on_hand_end_period': 323317.48,
              'committee_ids': ['C00431569', 'C00575795'],
              'coverage_end_date': '2016-12-31T00:00:00',
              'incumbent_challenge_full': 'Open seat',
              'party_full': 'DEMOCRATIC PARTY',
              'total_disbursements': 585580576.76,
              'total_receipts': 585699061.27},
             {'candidate_election_year': 2016,
              'candidate_id': 'P80001571',
              'candidate_name': 'TRUMP, DONALD J. / MICHAEL R. PENCE ',
              'candidate_pcc_id': 'C00580100',
             

<h4>Clean Data</h4>

In [232]:
def rename(name):
    #Some names are in format: Pres / Vice Pres - we only want President name
    name = name.split("/")[0]
    #President name in Last, First MI. format - we want First MI Last
    name_components = name.split(",")
    new_name = name_components[-1] + " " + name_components[0]
    return new_name.upper().strip()

#Apply the renaming funtion to the dataframe
candidate_finance["candidate_name"] = candidate_finance["candidate_name"].apply(rename)
candidate_finance


,candidate_election_year,candidate_id,candidate_name,party_full,total_receipts,total_disbursements,cash_on_hand_end_period
0,2000,P00003335,GEORGE W BUSH,REPUBLICAN PARTY,1.936577e+08,1.864560e+08,7201734.18
1,2000,P80000912,AL GORE,DEMOCRATIC PARTY,1.331080e+08,1.203352e+08,12772827.00
2,2000,P60003852,STEVE FORBES,REPUBLICAN PARTY,8.604857e+07,8.605477e+07,0.00
3,2000,P80002801,JOHN S MCCAIN,REPUBLICAN PARTY,5.863623e+07,5.854997e+07,86262.00
4,2000,P80000516,BILL BRADLEY,DEMOCRATIC PARTY,5.115617e+07,5.042141e+07,775203.00
...,...,...,...,...,...,...,...
95,2016,P60007671,MARTIN JOSEPH O'MALLEY,DEMOCRATIC PARTY,6.377550e+06,6.285182e+06,92368.71
96,2016,P60007697,LINDSEY O GRAHAM,REPUBLICAN PARTY,6.010010e+06,6.010010e+06,0.00
97,2016,P80003478,MIKE HUCKABEE,REPUBLICAN PARTY,4.335975e+06,4.330418e+06,5557.58
98,2016,P20002721,RICHARD J. SANTORUM,REPUBLICAN PARTY,2.296044e+06,2.312075e+06,110242.76


<h2>Election Result Data, CSV Extraction</h2>

<h4>Read CSV and Populate a DataFrame</h4>

In [233]:
election_csv = f"{year}_PopularVote.csv"



path = os.path.join("Resources", election_csv)
election_df = pd.read_csv(path, encoding="ISO-8859-1")



<h4>Clean Data</h4>

In [234]:
election_df = election_df.rename(columns = {
    "Candidate (Party Label)": "name"
})
election_df = election_df[["name", "Popular Vote Total", "Percent of Popular Vote"]]
election_df["year"] = 2000
election_df = election_df.dropna()

def conv_name(long_name):
    #Some data in FOrmat Name (Party) - we just want name
    short_name = long_name.split('(')[0]
    #MI is followed by ".", we want to remove that
    short_name = short_name.replace('.', '')
    return short_name.upper().strip()

#apply the name cleaning to the dataframe
election_df["name"] = election_df["name"].apply(conv_name)
election_df.head(100)




,name,Popular Vote Total,Percent of Popular Vote,year
1,AL GORE,"50,999,897",48.38,2000
2,GEORGE W BUSH,"50,456,002",47.87,2000
3,RALPH NADER,"2,882,955",2.74,2000
4,PATRICK J BUCHANAN,"448,895",0.42,2000
5,HARRY BROWNE,"384,431",0.36,2000
6,HOWARD PHILLIPS,"98,020",0.09,2000
7,JOHN S HAGELIN,"83,714",0.08,2000
8,WRITE-IN,"20,767",0.02,2000
9,"JAMES E HARRIS, JR","7,378",0.01,2000
10,L NEIL SMITH,"5,775",0.00,2000


<h4>Append folling year data to DataFrame</h4?

In [235]:
for yr in range(year + 4, 2017, 4):
    path = os.path.join("Resources", f"{yr}_PopularVote.csv")
    temp_df = pd.read_csv(path, encoding="ISO-8859-1")
    if yr == 2004: #2004 data is formatted differently
        temp_df = temp_df.rename(columns = {
            "Candidate": "name"
        })
        temp_df = temp_df[["name", "Popular Vote Total", "Percent of Popular Vote"]]
        temp_df = temp_df.dropna()
        temp_df["name"] = temp_df["name"].apply(conv_name)
        temp_df["year"] = yr
        election_df = election_df.append(temp_df, ignore_index=True)

    else:
        temp_df = temp_df.rename(columns = {
            "Candidate (Party Label)": "name"
        })
        temp_df = temp_df[["name", "Popular Vote Total", "Percent of Popular Vote"]]
        temp_df = temp_df.dropna()
        temp_df["name"] = temp_df["name"].apply(conv_name)
        temp_df["year"] = yr
        election_df = election_df.append(temp_df, ignore_index=True)
        election_df = election_df.dropna()

election_df = election_df.rename(columns = {"name": "Name", "Popular Vote Total": "Votes", "Percent of Popular Vote": "VotePct"})

df = pd.DataFrame(election_df)
df.head(100)








,Name,Votes,VotePct,year
0,AL GORE,"50,999,897",48.38,2000
1,GEORGE W BUSH,"50,456,002",47.87,2000
2,RALPH NADER,"2,882,955",2.74,2000
3,PATRICK J BUCHANAN,"448,895",0.42,2000
4,HARRY BROWNE,"384,431",0.36,2000
...,...,...,...,...
95,GARY JOHNSON,"4,489,341",3.28%,2016
96,JILL STEIN,"1,457,218",1.07%,2016
97,EVAN MCMULLIN,"731,991",0.54%,2016
98,WRITE-IN,"698,990",0.51%,2016


In [236]:
df.to_csv("popular_votes_years.csv")
df.to_json("popular_votes_years.json")


<h2>Store DataFrames into SQL Database Tables</h2>

In [237]:
conn = sqlite3.connect('ETL_project.db')
c = conn.cursor()

c.execute("DROP TABLE IF EXISTS Finance")
c.execute('CREATE TABLE Finance (candidate_election_year number, candidate_id text, candidate_name text, party_full text, total_receipts number, total_disbursements number, cash_on_hand_end_period number)')
conn.commit()

candidate_finance.to_sql('Finance', conn, if_exists='replace', index = False)


c.execute("SELECT * FROM Finance WHERE candidate_election_year = 2012")

print("=========================")
print("Financial Data for 2012")
for row in c.fetchall():
    print (row)
################################################################
c.execute("DROP TABLE IF EXISTS Votes")
c.execute('CREATE TABLE Votes (Name text, Votes int, VotePct text, year int)')
conn.commit()

election_df.to_sql('Votes', conn, if_exists='replace', index = False)



c.execute("SELECT * FROM Votes WHERE year = 2012")



print("=========================")
print("Vote Data for 2012")
for row in c.fetchall():
    print (row)
    


Financial Data for 2012
(2012, 'P80003338', 'BARACK OBAMA', 'DEMOCRATIC PARTY', 742601913.88, 757574968.34, 3299312.93)
(2012, 'P80003353', 'MITT  ROMNEY', 'REPUBLICAN PARTY', 483642897.19, 483292209.77, 383275.27)
(2012, 'P80000748', 'RON PAUL', 'REPUBLICAN PARTY', 41064074.11, 40495369.56, 1147671.75)
(2012, 'P60003654', 'NEWT GINGRICH', 'REPUBLICAN PARTY', 24976120.46, 24900556.09, 75564.37)
(2012, 'P20002721', 'RICHARD J. SANTORUM', 'REPUBLICAN PARTY', 23552730.52, 23426456.81, 126273.71)
(2012, 'P20003281', 'JAMES R (RICK) PERRY', 'REPUBLICAN PARTY', 20593581.15, 20199068.69, 394512.46)
(2012, 'P00003608', 'HERMAN CAIN', 'REPUBLICAN PARTY', 16917976.94, 16913859.48, 4117.46)
(2012, 'P20002184', 'RAPHAEL HERMAN', 'DEMOCRATIC PARTY', 16414018.0, 16414018.0, 0.0)
(2012, 'P20003067', 'JON HUNTSMAN', 'REPUBLICAN PARTY', 9173365.13, 9173365.13, 0.0)
(2012, 'P20002556', 'TIMOTHY PAWLENTY', 'REPUBLICAN PARTY', 5965502.16, 5965502.16, 0.0)
(2012, 'P20002671', 'GARY EARL JOHNSON', 'LIBERTAR

<h4>Practice putting SQL back into Dataframe</h4>

In [242]:
c.execute('''  
SELECT candidate_name, votes, year
FROM Finance
INNER JOIN Votes ON Finance.candidate_name = Votes.Name AND candidate_election_year = year
WHERE Finance.candidate_election_year = 2012
;
          ''')

# max_receipts = pd.DataFrame   
results = pd.DataFrame(c.fetchall(), columns=['candidate_name', "total_receipts", "year"]) 
results.head(50)


,candidate_name,total_receipts,year
0,BARACK OBAMA,"65,915,795",2012
1,JILL STEIN,"469,627",2012
